In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tszip

In [2]:
regions = [
    ("ORF1ab", 266, 21555),
    ("S", 21563, 25384),
    ("ORF3", 25393, 26220),
    ("E", 26245, 26472),
    ("M", 26523, 27191),
    ("ORF6", 27202, 27387),
    ("ORF7a", 27394, 27759),
    ("ORF7b", 27756, 27887),
    ("ORF8", 27894, 28259),
    ("N", 28274, 29533),
    ("ORF10", 29558, 29674)]

def get_genome_region(pos): 
    for label, start, end in regions:
        if start <= pos <= end:
            return label
    return "non-coding"


ORF1AB_regions = [
    ("NSP1", 266, 805),
    ("NSP2", 806, 2719),
    ("NSP3", 2720, 8554),
    ("NSP4", 8555, 10054),
    ("NSP5", 10055, 10972),
    ("NSP6", 10973, 11842),
    ("NSP7", 11843, 12091),
    ("NSP8", 12092, 12685),
    ("NSP9", 12686, 13024),
    ("NSP10", 13025, 13441),
    ("NSP12", 13441, 16236),  # frameshift happens here
    ("NSP13", 16237, 18039),
    ("NSP14", 18040, 19620),
    ("NSP15", 19621, 20658),
    ("NSP16", 20659, 21552)
]

def get_ORF1AB_region(pos):
    for label, start, end in ORF1AB_regions:
        if start <= pos <= end:
            return label
    return ""


spike_subregions = [
    ("S1-NTD-signal peptide", 21563, 21601),
    ("S1-NTD", 21563, 22475),
    ("S1-RBD", 22517, 23185),
    ("S1/2 cleavage site", 22517, 23185),
    ("S1-other", 21563, 23617),
    ("S2-Fusion peptide", 23924, 23980),
    ("S2-HR1", 24296, 24514),
    ("HR2", 25049, 25201),
    ("TM domain", 25202, 25273),
    ("CTD", 25274, 25383),
    ("S2-other", 23618, 25383)]

def get_spike_domain(pos):
    for label, start, end in spike_subregions:
        if start <= pos <= end:
            return label
    return ""

In [3]:
data_dir = Path("../data")

In [4]:
ts_file = data_dir / "sc2ts_viridian_v1.2.trees.tsz"
ts = tszip.load(ts_file)

In [5]:
events_file = data_dir / "deletion_events.csv"
events_df = pd.read_csv(events_file, index_col=0)
events_df["region"] = events_df["start"].apply(get_genome_region)
events_df["ORF1AB_region"] = events_df["start"].apply(get_ORF1AB_region)
events_df["spike_region"] = events_df["start"].apply(get_spike_domain)
events_df["pango"] = events_df["node"].apply(lambda x: ts.node(x).metadata["pango"])
events_df["deletion"] = events_df.apply(lambda row: tuple(range(row["start"], row["start"] + row["length"])), axis=1)
events_df["sc2ts_freq"] = np.round(events_df["max_inheritors"] / ts.num_samples * 100, decimals=4)
events_df = events_df.sort_values(by="min_inheritors", ascending=False)
events_df

,start,node,length,max_inheritors,min_inheritors,region,ORF1AB_region,spike_region,pango,deletion,sc2ts_freq
24047,28271,1436808,1,1118363,1118363,non-coding,,,B.1,"(28271,)",45.0561
21826,28248,220186,6,1114669,1114326,ORF8,,,B.1.617.2,"(28248, 28249, 28250, 28251, 28252, 28253)",44.9073
18802,22029,200039,6,1112504,1112033,S,,S1-NTD,B.1.617.2,"(22029, 22030, 22031, 22032, 22033, 22034)",44.8200
15018,11288,1436802,4,871782,871762,ORF1ab,NSP6,,B.1.1.529,"(11288, 11289, 11290, 11291)",35.1220
15019,28362,1436802,9,871774,850272,N,,,B.1.1.529,"(28362, 28363, 28364, 28365, 28366, 28367, 283...",35.1216
...,...,...,...,...,...,...,...,...,...,...,...
40623,29779,1078019,1,1,1,non-coding,,,BA.1.1,"(29779,)",0.0000
40622,29774,1078019,1,1,1,non-coding,,,BA.1.1,"(29774,)",0.0000
40621,29769,1078019,1,1,1,non-coding,,,BA.1.1,"(29769,)",0.0000
40620,29762,1078019,1,1,1,non-coding,,,BA.1.1,"(29762,)",0.0000


In [6]:
# Include only deletions at deletion remap sites.
# For now, just subset deletions based on these sites.
# This step won't be needed after
# https://github.com/jeromekelleher/sc2ts-paper/issues/809
remap_sites_file = "../arg_postprocessing/sc2ts_v1_2023-02-21_pr_pp_sites_to_remap.csv"
remap_df = pd.read_csv(remap_sites_file)
remap_df = remap_df[remap_df.reason == "deletion"].reset_index(drop=True)
remap_df.head(6)

,position,reason
0,686,deletion
1,687,deletion
2,688,deletion
3,689,deletion
4,690,deletion
5,691,deletion


In [7]:
events_df = events_df[events_df["start"].isin(remap_df["position"])].reset_index(drop=True)
events_df

,start,node,length,max_inheritors,min_inheritors,region,ORF1AB_region,spike_region,pango,deletion,sc2ts_freq
0,28271,1436808,1,1118363,1118363,non-coding,,,B.1,"(28271,)",45.0561
1,28248,220186,6,1114669,1114326,ORF8,,,B.1.617.2,"(28248, 28249, 28250, 28251, 28252, 28253)",44.9073
2,22029,200039,6,1112504,1112033,S,,S1-NTD,B.1.617.2,"(22029, 22030, 22031, 22032, 22033, 22034)",44.8200
3,11288,1436802,4,871782,871762,ORF1ab,NSP6,,B.1.1.529,"(11288, 11289, 11290, 11291)",35.1220
4,28362,1436802,9,871774,850272,N,,,B.1.1.529,"(28362, 28363, 28364, 28365, 28366, 28367, 283...",35.1216
...,...,...,...,...,...,...,...,...,...,...,...
11369,686,1014849,9,1,1,ORF1ab,NSP1,,BA.1,"(686, 687, 688, 689, 690, 691, 692, 693, 694)",0.0000
11370,686,1014562,9,1,1,ORF1ab,NSP1,,BA.1.15,"(686, 687, 688, 689, 690, 691, 692, 693, 694)",0.0000
11371,686,1014325,9,1,1,ORF1ab,NSP1,,BA.1.1,"(686, 687, 688, 689, 690, 691, 692, 693, 694)",0.0000
11372,686,1013857,9,1,1,ORF1ab,NSP1,,BA.1.1,"(686, 687, 688, 689, 690, 691, 692, 693, 694)",0.0000


### Major deletion events in the ARG

In [12]:
major_events_df = events_df[events_df["max_inheritors"] >= 10_000].reset_index(drop=True)
major_events_df

,start,node,length,max_inheritors,min_inheritors,region,ORF1AB_region,spike_region,pango,deletion,sc2ts_freq
0,28271,1436808,1,1118363,1118363,non-coding,,,B.1,"(28271,)",45.0561
1,28248,220186,6,1114669,1114326,ORF8,,,B.1.617.2,"(28248, 28249, 28250, 28251, 28252, 28253)",44.9073
2,22029,200039,6,1112504,1112033,S,,S1-NTD,B.1.617.2,"(22029, 22030, 22031, 22032, 22033, 22034)",44.8200
3,11288,1436802,4,871782,871762,ORF1ab,NSP6,,B.1.1.529,"(11288, 11289, 11290, 11291)",35.1220
4,28362,1436802,9,871774,850272,N,,,B.1.1.529,"(28362, 28363, 28364, 28365, 28366, 28367, 283...",35.1216
5,21633,822854,9,532072,531990,S,,S1-NTD,BA.2,"(21633, 21634, 21635, 21636, 21637, 21638, 216...",21.4359
6,11292,822854,5,531894,531867,ORF1ab,NSP6,,BA.2,"(11292, 11293, 11294, 11295, 11296)",21.4287
7,6513,851246,3,340757,340751,ORF1ab,NSP3,,BA.1,"(6513, 6514, 6515)",13.7283
8,11283,851246,5,339867,339865,ORF1ab,NSP6,,BA.1,"(11283, 11284, 11285, 11286, 11287)",13.6924
9,21765,851246,6,339681,339472,S,,S1-NTD,BA.1,"(21765, 21766, 21767, 21768, 21769, 21770)",13.6849


Table 1: Major deletion events which have more than 10k inheritors. Eight are located in the N-terminal domain of Spike, a previously noted site for key deletions. Many of these nodes represent the origins of major VOCs, for example, the 851246 node which represents the origin of BA.1, and the 86456 node which represents the origin of Alpha. All these deletions are known to occur in Alpha, Delta, or Omicron.

Note: The 4-base deletion and two 5-base deletions in ORF1ab are likely artefacts of our naive deletion remapping procedure. There are two 9-base deletions in the alignments, one common in BA.1 samples and the other in BA.2 samples. If the alignments were correct, the 4 deleted bases at 11,288 are shared, so they were pushed up above the split of BA.1 and BA.2 in the ARG. Alternatively, it is possible that the alignments are wrong and misleading, in which case there is a single 9-base deletion at 11,288 followed by a T>G mutation at 11,896, and the mutation caused the aligner to move the deletion to 11,283.

In [17]:
# Citations for lineage-defining deletions.
voc_dels = {
    # Alpha
    # https://virological.org/t/preliminary-genomic-characterisation-of-an-emergent-sars-cov-2-lineage-in-the-uk-defined-by-a-novel-set-of-spike-mutations/563
    "B.1.1.7": {11288: 9, 21765: 6, 21991: 3},
    # Delta
    # https://www.medrxiv.org/content/10.1101/2021.08.05.21261642v2.full
    "B.1.617.2": {22029: 6, 28248: 6},
    # Omicron
    # https://github.com/cov-lineages/pango-designation/issues/361
    "B.1.1.529": {11288: 9, 28362: 9},
    "BA.1": {6513: 3, 21765: 6, 21987:9, 22194: 3},
    "BA.2": {21633: 9},
    # REF
    "BA.4": {21765: 6},
}

voc_refs = {
    "B.1.1.7": "A",
    "B.1.617.2": "B",
    "B.1.1.529": "C",
    "BA.1": "D",
    "BA.2": "D",
    "BA.4": "E",
}

who_pango = {
    "B.1.1.7": "B.1.1.7 (Alpha)",
    "B.1.617.2": "B.1.617.2 (Delta)",
    "B.1.1.529": "B.1.1.529 (Omicron)",
    "BA.1": "BA.1 (Omicron)",
    "BA.2": "BA.2 (Omicron)",
    "BA.4": "BA.4 (Omicron)",
    "BA.5.2.1": "BA.5.2.1 (Omicron)",
}

latex_tab_major_dels = r'\begin{table}' + "\n" + \
r'\centering' + "\n" + \
r'\caption{Major deletion events in the ARG.}' + "\n" + \
r'\label{tab:major_dels}' + "\n" + \
r'\begin{tabular}{rrcrcrrc}' + "\n" + \
r'\toprule' + "\n" + \
r'Start & Length & Region & Node & Pango lineage & Count & Frequency (\%) & Ref\\' + "\n" + \
r'\midrule' + "\n"

for row in major_events_df.itertuples():
    sc2ts_count = f"{int(row.max_inheritors)}"
    sc2ts_freq = f"{row.sc2ts_freq:.4f}"
    region = row.region + " / " + row.ORF1AB_region if row.ORF1AB_region != "" else row.region
    region += " / " + row.spike_region if row.spike_region != "" else ""
    pango = who_pango[row.pango] if row.pango in who_pango else row.pango
    if (row.pango in voc_dels) and\
        (row.start in voc_dels[row.pango]) and\
        (voc_dels[row.pango][row.start] == row.length):
        ref = voc_refs[row.pango]
    else:
        ref = ""
    latex_tab_major_dels += f"{row.start} & {row.length} & {region} & " + \
        f"{row.node} & {pango} & " + \
        f"{sc2ts_count} & {sc2ts_freq} & {ref}" + r' \\' + "\n"

latex_tab_major_dels += r'\bottomrule' + "\n" + \
r'\end{tabular}' + "\n" + \
r'\end{table}'

print(latex_tab_major_dels)

\begin{table}
\centering
\caption{Major deletion events in the ARG.}
\label{tab:major_dels}
\begin{tabular}{rrcrcrrc}
\toprule
Start & Length & Region & Node & Pango lineage & Count & Frequency (\%) & Ref\\
\midrule
28271 & 1 & non-coding & 1436808 & B.1 & 1118363 & 45.0561 &  \\
28248 & 6 & ORF8 & 220186 & B.1.617.2 (Delta) & 1114669 & 44.9073 & B \\
22029 & 6 & S / S1-NTD & 200039 & B.1.617.2 (Delta) & 1112504 & 44.8200 & B \\
11288 & 4 & ORF1ab / NSP6 & 1436802 & B.1.1.529 (Omicron) & 871782 & 35.1220 &  \\
28362 & 9 & N & 1436802 & B.1.1.529 (Omicron) & 871774 & 35.1216 & C \\
21633 & 9 & S / S1-NTD & 822854 & BA.2 (Omicron) & 532072 & 21.4359 & D \\
11292 & 5 & ORF1ab / NSP6 & 822854 & BA.2 (Omicron) & 531894 & 21.4287 &  \\
6513 & 3 & ORF1ab / NSP3 & 851246 & BA.1 (Omicron) & 340757 & 13.7283 & D \\
11283 & 5 & ORF1ab / NSP6 & 851246 & BA.1 (Omicron) & 339867 & 13.6924 &  \\
21765 & 6 & S / S1-NTD & 851246 & BA.1 (Omicron) & 339681 & 13.6849 & D \\
21987 & 9 & S / S1-NTD & 851246

### Highly recurrent deletions in the ARG

In [18]:
rec_dels_df = events_df[["start", "length", "region", "ORF1AB_region", "spike_region"]]\
    .value_counts()\
    .reset_index(name="occurrences")[:20]
rec_dels_df

,start,length,region,ORF1AB_region,spike_region,occurrences
0,686,9,ORF1ab,NSP1,,6436
1,21991,3,S,,S1-NTD,2088
2,21765,6,S,,S1-NTD,482
3,22029,6,S,,S1-NTD,275
4,22194,3,S,,S1-NTD,200
5,6513,3,ORF1ab,NSP3,,181
6,21765,2,S,,S1-NTD,140
7,21995,1,S,,S1-NTD,90
8,21988,3,S,,S1-NTD,78
9,21987,7,S,,S1-NTD,78


Table 2: Highly recurrent deletions in the ARG. The majority of these recurrent deletions are located in the non-coding region, but some are in coding regions such as 21991-21993 and 22013-22018 in Spike. These are known recurrent deletions in S1-NTD: 6-base deletion at 21,765, 3-base deletion at 21,991, and 3-base deletion 22,194. See Figure 2 (RDR1 to RDR3) of McCarthy et al. (2021)[https://pmc.ncbi.nlm.nih.gov/articles/PMC7971772/]. The out-of-frame deletions (e.g., the 2-base deletion at 21,765 S1-NTD) are likely artefacts caused by
sequencing errors or sequence misalignments.

In [20]:
latex_tab_rec_dels = r'\begin{table}' + "\n" + \
r'\centering' + "\n" + \
r'\caption{Highly recurrent deletions.}' + "\n" + \
r'\label{tab:rec_dels}' + "\n" + \
r'\begin{tabular}{rrcr}' + "\n" + \
r'\toprule' + "\n" + \
r'Start & Length & Region & Occurrences & Ref\\' + "\n" + \
r'\midrule' + "\n"

for row in rec_dels_df.itertuples():
    region = row.region + " / " + row.ORF1AB_region if row.ORF1AB_region != "" else row.region
    region += " / " + row.spike_region if row.spike_region != "" else ""
    latex_tab_rec_dels += f"{row.start} & {row.length} & {region} & {row.occurrences} \\" + "\n"

latex_tab_rec_dels += r'\bottomrule' + "\n" + \
r'\end{tabular}' + "\n" + \
r'\end{table}'

print(latex_tab_rec_dels)

\begin{table}
\centering
\caption{Highly recurrent deletions.}
\label{tab:rec_dels}
\begin{tabular}{rrcr}
\toprule
Start & Length & Region & Occurrences & Ref\\
\midrule
686 & 9 & ORF1ab / NSP1 & 6436 \
21991 & 3 & S / S1-NTD & 2088 \
21765 & 6 & S / S1-NTD & 482 \
22029 & 6 & S / S1-NTD & 275 \
22194 & 3 & S / S1-NTD & 200 \
6513 & 3 & ORF1ab / NSP3 & 181 \
21765 & 2 & S / S1-NTD & 140 \
21995 & 1 & S / S1-NTD & 90 \
21988 & 3 & S / S1-NTD & 78 \
21987 & 7 & S / S1-NTD & 78 \
21993 & 3 & S / S1-NTD & 77 \
21987 & 9 & S / S1-NTD & 70 \
21994 & 2 & S / S1-NTD & 65 \
28370 & 1 & N & 61 \
11292 & 3 & ORF1ab / NSP6 & 48 \
21990 & 1 & S / S1-NTD & 46 \
21987 & 4 & S / S1-NTD & 46 \
21990 & 6 & S / S1-NTD & 45 \
686 & 7 & ORF1ab / NSP1 & 43 \
28252 & 2 & ORF8 & 41 \
\bottomrule
\end{tabular}
\end{table}


### Comparison with Li et al. (2023)

Li et al. (2023) https://pmc.ncbi.nlm.nih.gov/articles/PMC10580940/

In [ ]:
# https://figshare.com/articles/figure/Identifying_featured_indels_associated_with_SARS-CoV-2_fitness/21834957
tabs1_url = "https://figshare.com/ndownloader/files/40119535"
tabs1_df = pd.read_excel(tabs1_url)

tabs1_df.rename(columns={"Start": "start", "Length": "length"}, inplace=True)

tabs1_df["region"] = tabs1_df["start"].apply(get_genome_region)
tabs1_df["ORF1AB_region"] = tabs1_df["start"].apply(get_ORF1AB_region)
tabs1_df["spike_region"] = tabs1_df["start"].apply(get_spike_domain)

# "In total, 31,642,407 deletion records and 1,981,308 insertion records were detected 
# in 9,149,680 filtered genomic sequences after removing indel records in 5′ and 3′ untranslated regions (UTR), and 
# there were 26,765 different types of deletions and 21,054 different types of insertions (Table S1)."
num_samples_li = 9149680

tabs1_df["li_freq"] = np.round(tabs1_df["Count"] / num_samples_li * 100, decimals=4)
tabs1_df.head(5)

In [ ]:
# Check that these are all deletions in tab 1 of Table S1.
all(tabs1_df["Name"].str.startswith("Del_"))

In [ ]:
agg_dels_df = events_df.groupby(["start", "length"])\
.agg({
    'start': 'first',
    'length': 'first',
    'node': 'unique',
    'pango': 'unique',
    'max_inheritors': 'sum',
    'region': 'first',
    'ORF1AB_region': 'first',
    'spike_region': 'first',
})\
.sort_values(by='max_inheritors', ascending=False)\
.reset_index(drop=True)
agg_dels_df["agg_sc2ts_freq"] = np.round(agg_dels_df["max_inheritors"] / ts.num_samples * 100, decimals=4)
agg_dels_df.head(5)

In [ ]:
# Check one case.
sum(events_df[(events_df["start"] == 28271) & (events_df["length"] == 1)]['max_inheritors'])

In [ ]:
agg_dels_df = agg_dels_df.merge(
    tabs1_df,
    how="outer",    # NOTE
    left_on=["start", "length", "region", "ORF1AB_region", "spike_region"],
    right_on=["start", "length", "region", "ORF1AB_region", "spike_region"],
)[[
    "start", "length", "region", "ORF1AB_region", "spike_region",
    "max_inheritors", "agg_sc2ts_freq", "Count", "li_freq",
]]
agg_dels_df.head(5)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))

ax[0].scatter(
    x=agg_dels_df["agg_sc2ts_freq"],
    y=agg_dels_df["li_freq"],
    s=20,
    marker='x',
)
ax[0].set_xlim(-5, 60)
ax[0].set_ylim(-5, 60)
ax[0].set_xlabel("Sc2ts frequency (%)")
ax[0].set_ylabel("Li et al. (2023) frequency (%)")
for i, row in agg_dels_df.iterrows():
    ax[0].text(
        row["agg_sc2ts_freq"],
        row["li_freq"],
        str(row["start"]) + "_" + str(row["length"]),
        fontsize=8,
        alpha=0.8,
        ha='right',
        va='bottom',
    )
ax[0].axline((0, 0), slope=1, color='k', linestyle='--', lw=1, alpha=0.7)


ax[1].set_title("Zoomed")
ax[1].scatter(
    x=agg_dels_df["agg_sc2ts_freq"],
    y=agg_dels_df["li_freq"],
    s=20,
    marker='x',
)
ax[1].set_xlim(-0.1, 1)
ax[1].set_ylim(-0.1, 1)
ax[1].set_xlabel("Sc2ts frequency (%)")
ax[1].set_ylabel("Li et al. (2023) frequency (%)")
for i, row in agg_dels_df.iterrows():
    if row["agg_sc2ts_freq"] > 1.0 or row["li_freq"] > 1.0:
        continue
    ax[1].text(
        row["agg_sc2ts_freq"],
        row["li_freq"],
        str(row["start"]) + "_" + str(row["length"]),
        fontsize=8,
        alpha=0.8,
        ha='right',
        va='bottom',
    )
ax[1].axline((0, 0), slope=1, color='k', linestyle='--', lw=1, alpha=0.7);

In [ ]:
common_dels_df = agg_dels_df[
    (agg_dels_df["agg_sc2ts_freq"] >= 1.0) | (agg_dels_df["li_freq"] >= 1.0)
]\
.sort_values(by="agg_sc2ts_freq", ascending=False)\
.reset_index(drop=True)
common_dels_df

In [ ]:
latex_tab_common_dels = r'\begin{table}' + "\n" + \
r'\centering' + "\n" + \
r'\caption{Most commonly observed deletions in the ARG or Li et al. (2023).} ' + "\n" + \
r'\label{tab:common_dels}' + "\n" + \
r'\begin{tabular}{rrcrrrr}' + "\n" + \
r'\toprule' + "\n" + \
r'\multicolumn{3}{c}{} & \multicolumn{2}{c}{Sc2ts ARG} & \multicolumn{2}{c}{Li et al. (2023)} \\' + "\n" + \
r'\cmidrule(lr){4-5} \cmidrule(lr){6-7}' + "\n" + \
r'Start & Length & Region & Count & Frequency (\%) & Count & Frequency (\%) \\' + "\n" + \
r'\midrule' + "\n"

for row in common_dels_df.itertuples():
    region = row.region + " / " + row.ORF1AB_region if row.ORF1AB_region != "" else row.region
    region += " / " + row.spike_region if row.spike_region != "" else ""
    sc2ts_count = "N/A" if np.isnan(row.max_inheritors) else f"{int(row.max_inheritors)}"
    sc2ts_freq = "N/A" if np.isnan(row.agg_sc2ts_freq) else f"{row.agg_sc2ts_freq:.4f}"
    li_count = "N/A" if np.isnan(row.Count) else f"{int(row.Count)}"
    li_freq = "N/A" if np.isnan(row.li_freq) else f"{row.li_freq:.4f}"
    latex_tab_common_dels += f"{row.start} & {row.length} & {region} & " + \
        f"{sc2ts_count} & {sc2ts_freq} & {li_count} & {li_freq}" + r' \\' + "\n"

latex_tab_common_dels += r'\bottomrule' + "\n" + \
r'\end{tabular}' + "\n" + \
r'\end{table}'

print(latex_tab_common_dels)

Table (used to update remapped sites): Deletions commonly observed in the Sc2ts ARG and Li et al. (2023). These deletions occur at least 1% of the samples analyzed.